# CMU-MOSEI Preprocessing Script 处理MMSA中的MOSEI-lable数据
This script is used for generating RAW_PREPROCESSED data from the raw CMU-MOSEI data, you can download the raw dataset from:


In [1]:
import os, sys
from shutil import copyfile
import glob
import pickle

import pandas as pd
import numpy as np
import h5py
import cv2
from scipy.io import wavfile
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing

In [2]:
# label输入输出的基本路经
base_label_input_path = '/home/luocong/projects/MMSA1.0/Datasets/MOSEI/MOSEI-label.csv' # MOSEI_label path
base_label_output_path = '/home/luocong/projects/data/data/MOSEI_SEN_SPLIT' 
base_out_put_path = '/home/luocong/projects/MMSA1.0/Datasets/MOSEI'


# 视频片段的原路经和复制路经
base_video_seg_path = '/home/luocong/projects/data/data/MOSEI_RAW_PROCESSED/'
video_seg_out_path = '/home/luocong/projects/data/data/MOSEI_SEN_PROCESSED/'

base_path = '/home/luocong/projects/MMSA1.0/Datasets/SIMS/Raw'
sims_label_path = f'{base_path}/Labels/SIMS-label.csv'

# Number of jobs to run extraction in parallel
num_jobs = 32 

### Load Labels

In [3]:
# 打开csv文件
# Load data
dfs = []
for path in glob.glob(base_label_input_path):
    dfs.append(pd.read_csv(path))
# Generate table
df = pd.concat(dfs)
df.head(20)
# 没有等号

,video_id,clip_id,text,label,annotation,mode,label_by
0,-3g5yACwYnA,10,Key is part of the people that we use to solve...,1.000000,Positive,train,0
1,-3g5yACwYnA,13,They've been able to find solutions or at leas...,0.666667,Positive,train,0
2,-3g5yACwYnA,3,We're a huge user of adhesives for our operati...,0.000000,Neutral,train,0
3,-3g5yACwYnA,2,Key Polymer brings a technical aspect to our o...,0.000000,Neutral,train,0
4,-3g5yACwYnA,4,Key brings those types of aspects to a busines...,1.000000,Positive,train,0
5,-3g5yACwYnA,9,We have many new opportunities through the way...,0.666667,Positive,train,0
6,-3nNcZdcdvU,0,"Introduction Speaker: Dr. Erma Jean Sims, Sono...",0.333333,Positive,train,0
7,-3nNcZdcdvU,2,It's important for teachers to understand thei...,1.000000,Positive,train,0
8,-3nNcZdcdvU,5,Noncompliance can result in legal liability to...,0.000000,Neutral,train,0
9,-HwX2H8Z4hY,1,"A 1983 report, “A Nation at Risk,” argued that...",-0.666667,Negative,train,0


In [4]:
# 构造uttr_id
df['uttr_id'] = df.apply(lambda x: f"{str(x['video_id'])}_{x['clip_id']}" , axis='columns')
# 提取5列形成新表格
ldf = df[['uttr_id','text','label','annotation','mode']]
# 显示新表格
ldf.head(30)

,uttr_id,text,label,annotation,mode
0,-3g5yACwYnA_10,Key is part of the people that we use to solve...,1.000000,Positive,train
1,-3g5yACwYnA_13,They've been able to find solutions or at leas...,0.666667,Positive,train
2,-3g5yACwYnA_3,We're a huge user of adhesives for our operati...,0.000000,Neutral,train
3,-3g5yACwYnA_2,Key Polymer brings a technical aspect to our o...,0.000000,Neutral,train
4,-3g5yACwYnA_4,Key brings those types of aspects to a busines...,1.000000,Positive,train
5,-3g5yACwYnA_9,We have many new opportunities through the way...,0.666667,Positive,train
6,-3nNcZdcdvU_0,"Introduction Speaker: Dr. Erma Jean Sims, Sono...",0.333333,Positive,train
7,-3nNcZdcdvU_2,It's important for teachers to understand thei...,1.000000,Positive,train
8,-3nNcZdcdvU_5,Noncompliance can result in legal liability to...,0.000000,Neutral,train
9,-HwX2H8Z4hY_1,"A 1983 report, “A Nation at Risk,” argued that...",-0.666667,Negative,train


In [38]:
# 保存这个含有5项数据的新表格
# header=0不保留表头 header=1保留表头 index=False不保存索引id 保存到csv正常 xls异常
ldf.to_csv(f'{base_out_put_path}/new_label_Result.csv',header=1,index=False)

### 提取多模态数据

In [ ]:
if not os.path.exists(video_seg_out_path):
    os.makedirs(video_seg_out_path)

In [7]:
%%time
# Text and label
# 文本和标签

# 先构造一个空字典
#sims数据集上是三模态标签加上混合模态标签
# 原来mosei的label是一个发声视频片段的多分类标签

# 元的意思是原本的核心的 所以元数据就算多模态数据集的一个个样本点

# sims不需要开始时间点和结束时间点 视频片段已经分好了 彼此不一定连续
meta_dict = {}
i=0
for row in tqdm(ldf.itertuples()):
    #这里的id指的就是uttr_id 也就是vedio_seg_id
    vedio_seg_id, label, text = row[1], row[3], row[2] #list(row[4:8])
    

    # Store id, text, and labels to metadata buffer
    # 保存id，文本和标签到元数据数组
    text = text.replace('\n','')
#     print(vedio_seg_id, label, text)
#     break
        
    # 键是发声片段的id即vedio_id_seg_id 值也是一个字典 字典里有两个键值对 'text': text,'label': m_label
    meta_dict[vedio_seg_id] = {
        'text': text,
        'label': label
    }
    i = i+1
    print('已经处理了%d个'%i) 

# Save metadata
# 保存元数据为python的序列化类型
pickle.dump(meta_dict, open(f'{video_seg_out_path}/meta.pkl','wb'))

2418it [00:00, 22520.48it/s]

已经处理了1个
已经处理了2个
已经处理了3个
已经处理了4个
已经处理了5个
已经处理了6个
已经处理了7个
已经处理了8个
已经处理了9个
已经处理了10个
已经处理了11个
已经处理了12个
已经处理了13个
已经处理了14个
已经处理了15个
已经处理了16个
已经处理了17个
已经处理了18个
已经处理了19个
已经处理了20个
已经处理了21个
已经处理了22个
已经处理了23个
已经处理了24个
已经处理了25个
已经处理了26个
已经处理了27个
已经处理了28个
已经处理了29个
已经处理了30个
已经处理了31个
已经处理了32个
已经处理了33个
已经处理了34个
已经处理了35个
已经处理了36个
已经处理了37个
已经处理了38个
已经处理了39个
已经处理了40个
已经处理了41个
已经处理了42个
已经处理了43个
已经处理了44个
已经处理了45个
已经处理了46个
已经处理了47个
已经处理了48个
已经处理了49个
已经处理了50个
已经处理了51个
已经处理了52个
已经处理了53个
已经处理了54个
已经处理了55个
已经处理了56个
已经处理了57个
已经处理了58个
已经处理了59个
已经处理了60个
已经处理了61个
已经处理了62个
已经处理了63个
已经处理了64个
已经处理了65个
已经处理了66个
已经处理了67个
已经处理了68个
已经处理了69个
已经处理了70个
已经处理了71个
已经处理了72个
已经处理了73个
已经处理了74个
已经处理了75个
已经处理了76个
已经处理了77个
已经处理了78个
已经处理了79个
已经处理了80个
已经处理了81个
已经处理了82个
已经处理了83个
已经处理了84个
已经处理了85个
已经处理了86个
已经处理了87个
已经处理了88个
已经处理了89个
已经处理了90个
已经处理了91个
已经处理了92个
已经处理了93个
已经处理了94个
已经处理了95个
已经处理了96个
已经处理了97个
已经处理了98个
已经处理了99个
已经处理了100个
已经处理了101个
已经处理了102个
已经处理了103个
已经处理了104个
已经处理了105个
已经处理了106个
已经处理了107个
已经处理了108个
已经处理了109个
已经处理了110个
已经处理了111

7058it [00:00, 16413.81it/s]


已经处理了4780个
已经处理了4781个
已经处理了4782个
已经处理了4783个
已经处理了4784个
已经处理了4785个
已经处理了4786个
已经处理了4787个
已经处理了4788个
已经处理了4789个
已经处理了4790个
已经处理了4791个
已经处理了4792个
已经处理了4793个
已经处理了4794个
已经处理了4795个
已经处理了4796个
已经处理了4797个
已经处理了4798个
已经处理了4799个
已经处理了4800个
已经处理了4801个
已经处理了4802个
已经处理了4803个
已经处理了4804个
已经处理了4805个
已经处理了4806个
已经处理了4807个
已经处理了4808个
已经处理了4809个
已经处理了4810个
已经处理了4811个
已经处理了4812个
已经处理了4813个
已经处理了4814个
已经处理了4815个
已经处理了4816个
已经处理了4817个
已经处理了4818个
已经处理了4819个
已经处理了4820个
已经处理了4821个
已经处理了4822个
已经处理了4823个
已经处理了4824个
已经处理了4825个
已经处理了4826个
已经处理了4827个
已经处理了4828个
已经处理了4829个
已经处理了4830个
已经处理了4831个
已经处理了4832个
已经处理了4833个
已经处理了4834个
已经处理了4835个
已经处理了4836个
已经处理了4837个
已经处理了4838个
已经处理了4839个
已经处理了4840个
已经处理了4841个
已经处理了4842个
已经处理了4843个
已经处理了4844个
已经处理了4845个
已经处理了4846个
已经处理了4847个
已经处理了4848个
已经处理了4849个
已经处理了4850个
已经处理了4851个
已经处理了4852个
已经处理了4853个
已经处理了4854个
已经处理了4855个
已经处理了4856个
已经处理了4857个
已经处理了4858个
已经处理了4859个
已经处理了4860个
已经处理了4861个
已经处理了4862个
已经处理了4863个
已经处理了4864个
已经处理了4865个
已经处理了4866个
已经处理了4867个
已经处理了4868个
已经处理了4869个
已经处理了4870

12196it [00:00, 19937.39it/s]

...


已经处理了12908个
已经处理了12909个
已经处理了12910个
已经处理了12911个
已经处理了12912个
已经处理了12913个
已经处理了12914个
已经处理了12915个
已经处理了12916个
已经处理了12917个
已经处理了12918个
已经处理了12919个
已经处理了12920个
已经处理了12921个
已经处理了12922个
已经处理了12923个
已经处理了12924个
已经处理了12925个
已经处理了12926个
已经处理了12927个
已经处理了12928个
已经处理了12929个
已经处理了12930个
已经处理了12931个
已经处理了12932个
已经处理了12933个
已经处理了12934个
已经处理了12935个
已经处理了12936个
已经处理了12937个
已经处理了12938个
已经处理了12939个
已经处理了12940个
已经处理了12941个
已经处理了12942个
已经处理了12943个
已经处理了12944个
已经处理了12945个
已经处理了12946个
已经处理了12947个
已经处理了12948个
已经处理了12949个
已经处理了12950个
已经处理了12951个
已经处理了12952个
已经处理了12953个
已经处理了12954个
已经处理了12955个
已经处理了12956个
已经处理了12957个
已经处理了12958个
已经处理了12959个
已经处理了12960个
已经处理了12961个
已经处理了12962个
已经处理了12963个
已经处理了12964个
已经处理了12965个
已经处理了12966个
已经处理了12967个
已经处理了12968个
已经处理了12969个
已经处理了12970个
已经处理了12971个
已经处理了12972个
已经处理了12973个
已经处理了12974个
已经处理了12975个
已经处理了12976个
已经处理了12977个
已经处理了12978个
已经处理了12979个
已经处理了12980个
已经处理了12981个
已经处理了12982个
已经处理了12983个
已经处理了12984个
已经处理了12985个
已经处理了12986个
已经处理了12987个
已经处理了12988个
已经处理了12989个
已经处理了12990个
已经处

21867it [00:01, 20606.59it/s]

已经处理了17903个
已经处理了17904个
已经处理了17905个
已经处理了17906个
已经处理了17907个
已经处理了17908个
已经处理了17909个
已经处理了17910个
已经处理了17911个
已经处理了17912个
已经处理了17913个
已经处理了17914个
已经处理了17915个
已经处理了17916个
已经处理了17917个
已经处理了17918个
已经处理了17919个
已经处理了17920个
已经处理了17921个
已经处理了17922个
已经处理了17923个
已经处理了17924个
已经处理了17925个
已经处理了17926个
已经处理了17927个
已经处理了17928个
已经处理了17929个
已经处理了17930个
已经处理了17931个
已经处理了17932个
已经处理了17933个
已经处理了17934个
已经处理了17935个
已经处理了17936个
已经处理了17937个
已经处理了17938个
已经处理了17939个
已经处理了17940个
已经处理了17941个
已经处理了17942个
已经处理了17943个
已经处理了17944个
已经处理了17945个
已经处理了17946个
已经处理了17947个
已经处理了17948个
已经处理了17949个
已经处理了17950个
已经处理了17951个
已经处理了17952个
已经处理了17953个
已经处理了17954个
已经处理了17955个
已经处理了17956个
已经处理了17957个
已经处理了17958个
已经处理了17959个
已经处理了17960个
已经处理了17961个
已经处理了17962个
已经处理了17963个
已经处理了17964个
已经处理了17965个
已经处理了17966个
已经处理了17967个
已经处理了17968个
已经处理了17969个
已经处理了17970个
已经处理了17971个
已经处理了17972个
已经处理了17973个
已经处理了17974个
已经处理了17975个
已经处理了17976个
已经处理了17977个
已经处理了17978个
已经处理了17979个
已经处理了17980个
已经处理了17981个
已经处理了17982个
已经处理了17983个
已经处理了17984个
已经处理了17985个
已经处理

22856it [00:01, 20226.12it/s]


已经处理了22365个
已经处理了22366个
已经处理了22367个
已经处理了22368个
已经处理了22369个
已经处理了22370个
已经处理了22371个
已经处理了22372个
已经处理了22373个
已经处理了22374个
已经处理了22375个
已经处理了22376个
已经处理了22377个
已经处理了22378个
已经处理了22379个
已经处理了22380个
已经处理了22381个
已经处理了22382个
已经处理了22383个
已经处理了22384个
已经处理了22385个
已经处理了22386个
已经处理了22387个
已经处理了22388个
已经处理了22389个
已经处理了22390个
已经处理了22391个
已经处理了22392个
已经处理了22393个
已经处理了22394个
已经处理了22395个
已经处理了22396个
已经处理了22397个
已经处理了22398个
已经处理了22399个
已经处理了22400个
已经处理了22401个
已经处理了22402个
已经处理了22403个
已经处理了22404个
已经处理了22405个
已经处理了22406个
已经处理了22407个
已经处理了22408个
已经处理了22409个
已经处理了22410个
已经处理了22411个
已经处理了22412个
已经处理了22413个
已经处理了22414个
已经处理了22415个
已经处理了22416个
已经处理了22417个
已经处理了22418个
已经处理了22419个
已经处理了22420个
已经处理了22421个
已经处理了22422个
已经处理了22423个
已经处理了22424个
已经处理了22425个
已经处理了22426个
已经处理了22427个
已经处理了22428个
已经处理了22429个
已经处理了22430个
已经处理了22431个
已经处理了22432个
已经处理了22433个
已经处理了22434个
已经处理了22435个
已经处理了22436个
已经处理了22437个
已经处理了22438个
已经处理了22439个
已经处理了22440个
已经处理了22441个
已经处理了22442个
已经处理了22443个
已经处理了22444个
已经处理了22445个
已经处理了22446个
已经处理了22447个
已经处理

In [37]:
# 提取训练集train_label
train_label_df=df[(df['mode']=='train')]
train_label_df=train_label_df[['uttr_id']]
train_label_df

,uttr_id
0,-3g5yACwYnA_10
1,-3g5yACwYnA_13
2,-3g5yACwYnA_3
3,-3g5yACwYnA_2
4,-3g5yACwYnA_4
...,...
16321,zwTrXwi54us_7
16322,zwTrXwi54us_6
16323,zwTrXwi54us_9
16324,zwTrXwi54us_8


In [42]:
# 保存训练集标签
train_label_df.to_csv(f'{base_label_output_path}/train_split.txt',header=0,index=False)

In [44]:
# 提取验证集 valid_label
valid_label_df=df[(df['mode']=='valid')]
valid_label_df=valid_label_df[['uttr_id']]
valid_label_df

,uttr_id
16326,--qXJuDtHPw_5
16327,-571d8cVauQ_0
16328,-571d8cVauQ_3
16329,-571d8cVauQ_5
16330,-I_e4mIh0yE_1
...,...
18192,zjYEBwXGD8I_6
18193,zjYEBwXGD8I_9
18194,zjYEBwXGD8I_8
18195,zk2jTlAtvSU_3


In [45]:
# 保存验证集标签
valid_label_df.to_csv(f'{base_label_output_path}/valid_split.txt',header=0,index=False)

In [46]:
# 提取测试集 test_label
test_label_df=df[(df['mode']=='test')]
test_label_df=test_label_df[['uttr_id']]
test_label_df

,uttr_id
18197,-6rXp3zJ3kc_9
18198,-6rXp3zJ3kc_8
18199,-6rXp3zJ3kc_3
18200,-9y-fZ3swSY_1
18201,-9y-fZ3swSY_0
...,...
22851,zhNksSReaQk_35
22852,zhNksSReaQk_34
22853,zhNksSReaQk_33
22854,zvZd3V5D5Ik_3


In [47]:
# 保存测试集标签
test_label_df.to_csv(f'{base_label_output_path}/test_split.txt',header=0,index=False)

In [49]:
# 提取正面文本 POS
positive_df=df[(df['annotation']=='Positive')]
positive_df=positive_df[['text']]
positive_df

,text
0,Key is part of the people that we use to solve...
1,They've been able to find solutions or at leas...
4,Key brings those types of aspects to a busines...
5,We have many new opportunities through the way...
6,"Introduction Speaker: Dr. Erma Jean Sims, Sono..."
...,...
22837,You can see all my carbonated water over there
22841,And you can see my cat calendar over there
22843,It's all black cats
22854,"If you're ready to strengthen your skills, whi..."


In [56]:
# 保存正面文本
positive_df.to_csv(f'{base_label_output_path}/pos.csv',header=0,index=False)

In [52]:
# 提取负面文本 POS
# 含0的负面
# 再取一次不含0的负面
negative_df=df[(df['annotation']=='Neutral')|(df['annotation']=='Negative')]
negative_df=negative_df[['text']]
negative_df

,text
2,We're a huge user of adhesives for our operati...
3,Key Polymer brings a technical aspect to our o...
8,Noncompliance can result in legal liability to...
9,"A 1983 report, “A Nation at Risk,” argued that..."
10,the state of American education
...,...
22849,"And the thing is, there are a/Autistic people ..."
22850,And in the process we hurt our intellectually ...
22851,"And yet, it's like, how a/Autistic people defe..."
22852,But the thing is that intelligence [scoffs] is...


In [57]:
# 保存负面文本
negative_df.to_csv(f'{base_label_output_path}/neg.csv',header=0,index=False)

In [59]:
# 提取出所有的uttr_id
uttr_id_df=df[['uttr_id']]
uttr_id_df

,uttr_id
0,-3g5yACwYnA_10
1,-3g5yACwYnA_13
2,-3g5yACwYnA_3
3,-3g5yACwYnA_2
4,-3g5yACwYnA_4
...,...
22851,zhNksSReaQk_35
22852,zhNksSReaQk_34
22853,zhNksSReaQk_33
22854,zvZd3V5D5Ik_3


In [63]:
import shutil

In [68]:
# 测试复制一个文件夹
# uttr_id = '-3g5yACwYnA_10'
# olddir = f'{base_video_seg_path}{uttr_id}'
# newdir = f'{video_seg_out_path}{uttr_id}'
# print(olddir)
# print(newdir)

/home/luocong/projects/data/data/MOSEI_RAW_PROCESSED/-3g5yACwYnA_10
/home/luocong/projects/data/data/MOSEI_SEN_PROCESSED/-3g5yACwYnA_10


In [69]:
# 复制视频片段到新文件夹
# 先测试复制一个文件夹下的所有内容 把该文件夹复制到制定路经
# if not os.path.exists(base_out_path):
#     os.makedirs(base_out_path)

In [70]:
# base_video_seg_path = '/home/luocong/projects/data/data/MOSEI_RAW_PROCESSED/'
# video_seg_out_path = '/home/luocong/projects/data/data/MOSEI_SEN_PROCESSED/'

In [74]:
# shutil.copy(olddir,newdir)
shutil.copytree(olddir,newdir)
# 复制成功

'/home/luocong/projects/data/data/MOSEI_SEN_PROCESSED/-3g5yACwYnA_10'

In [81]:
# 遍历uttr_id_df
i=1
for index, row in uttr_id_df.iterrows():
#     print(row)
#     print(row['uttr_id']) # 输出每行的索引值
    uttr_id = row['uttr_id']
    olddir = f'{base_video_seg_path}{uttr_id}'
    newdir = f'{video_seg_out_path}{uttr_id}'
    shutil.copytree(olddir,newdir)
    print(f'已复制{uttr_id}文件夹,总共已复制{i}个')
    i = i + 1
#     break

已复制-3g5yACwYnA_10文件夹,总共已复制1个
已复制-3g5yACwYnA_13文件夹,总共已复制2个
已复制-3g5yACwYnA_3文件夹,总共已复制3个
已复制-3g5yACwYnA_2文件夹,总共已复制4个
已复制-3g5yACwYnA_4文件夹,总共已复制5个
已复制-3g5yACwYnA_9文件夹,总共已复制6个
已复制-3nNcZdcdvU_0文件夹,总共已复制7个
已复制-3nNcZdcdvU_2文件夹,总共已复制8个
已复制-3nNcZdcdvU_5文件夹,总共已复制9个
已复制-HwX2H8Z4hY_1文件夹,总共已复制10个
已复制-HwX2H8Z4hY_0文件夹,总共已复制11个
已复制-HwX2H8Z4hY_3文件夹,总共已复制12个
已复制-HwX2H8Z4hY_2文件夹,总共已复制13个
已复制-HwX2H8Z4hY_5文件夹,总共已复制14个
已复制-HwX2H8Z4hY_4文件夹,总共已复制15个
已复制-HwX2H8Z4hY_7文件夹,总共已复制16个
已复制-HwX2H8Z4hY_6文件夹,总共已复制17个
已复制-HwX2H8Z4hY_9文件夹,总共已复制18个
已复制-HwX2H8Z4hY_8文件夹,总共已复制19个
已复制-NFrJFQijFE_1文件夹,总共已复制20个
已复制-NFrJFQijFE_8文件夹,总共已复制21个
已复制-NFrJFQijFE_2文件夹,总共已复制22个
已复制-NFrJFQijFE_0文件夹,总共已复制23个
已复制-THoVjtIkeU_11文件夹,总共已复制24个
已复制-THoVjtIkeU_10文件夹,总共已复制25个
已复制-THoVjtIkeU_12文件夹,总共已复制26个
已复制-THoVjtIkeU_1文件夹,总共已复制27个
已复制-THoVjtIkeU_0文件夹,总共已复制28个
已复制-THoVjtIkeU_2文件夹,总共已复制29个
已复制-THoVjtIkeU_6文件夹,总共已复制30个
已复制-THoVjtIkeU_8文件夹,总共已复制31个
已复制-UuX1xuaiiE_1文件夹,总共已复制32个
已复制-UuX1xuaiiE_0文件夹,总共已复制33个
已复制-UuX1xuaiiE_3文件夹,总共已复制34个
已复制-UuX1xuaiiE_2文件

已复制0vXaXWx7Rvo_9文件夹,总共已复制338个
已复制0vXaXWx7Rvo_8文件夹,总共已复制339个
已复制0zl_GDVzykw_1文件夹,总共已复制340个
已复制0zl_GDVzykw_5文件夹,总共已复制341个
已复制100499_1文件夹,总共已复制342个
已复制100499_0文件夹,总共已复制343个
已复制100499_3文件夹,总共已复制344个
已复制100499_2文件夹,总共已复制345个
已复制100499_5文件夹,总共已复制346个
已复制100499_4文件夹,总共已复制347个
已复制100499_6文件夹,总共已复制348个
已复制101851_11文件夹,总共已复制349个
已复制101851_10文件夹,总共已复制350个
已复制101851_13文件夹,总共已复制351个
已复制101851_12文件夹,总共已复制352个
已复制101851_20文件夹,总共已复制353个
已复制101851_14文件夹,总共已复制354个
已复制101851_17文件夹,总共已复制355个
已复制101851_16文件夹,总共已复制356个
已复制101851_19文件夹,总共已复制357个
已复制101851_18文件夹,总共已复制358个
已复制101851_1文件夹,总共已复制359个
已复制101851_0文件夹,总共已复制360个
已复制101851_3文件夹,总共已复制361个
已复制101851_2文件夹,总共已复制362个
已复制101851_5文件夹,总共已复制363个
已复制101851_4文件夹,总共已复制364个
已复制101851_7文件夹,总共已复制365个
已复制101851_6文件夹,总共已复制366个
已复制101851_9文件夹,总共已复制367个
已复制101851_15文件夹,总共已复制368个
已复制101851_8文件夹,总共已复制369个
已复制102168_1文件夹,总共已复制370个
已复制102168_0文件夹,总共已复制371个
已复制102168_3文件夹,总共已复制372个
已复制102168_2文件夹,总共已复制373个
已复制102168_4文件夹,总共已复制374个
已复制102184_11文件夹,总共已复制375个
已复制102184_10文件夹,总共

已复制122842_4文件夹,总共已复制703个
已复制122842_7文件夹,总共已复制704个
已复制122842_6文件夹,总共已复制705个
已复制122842_9文件夹,总共已复制706个
已复制122842_8文件夹,总共已复制707个
已复制125708_0文件夹,总共已复制708个
已复制125730_11文件夹,总共已复制709个
已复制125730_10文件夹,总共已复制710个
已复制125730_13文件夹,总共已复制711个
已复制125730_12文件夹,总共已复制712个
已复制125730_15文件夹,总共已复制713个
已复制125730_14文件夹,总共已复制714个
已复制125730_1文件夹,总共已复制715个
已复制125730_0文件夹,总共已复制716个
已复制125730_3文件夹,总共已复制717个
已复制125730_2文件夹,总共已复制718个
已复制125730_5文件夹,总共已复制719个
已复制125730_4文件夹,总共已复制720个
已复制125730_7文件夹,总共已复制721个
已复制125730_6文件夹,总共已复制722个
已复制125730_9文件夹,总共已复制723个
已复制125730_8文件夹,总共已复制724个
已复制125845_10文件夹,总共已复制725个
已复制125845_1文件夹,总共已复制726个
已复制125845_0文件夹,总共已复制727个
已复制125845_3文件夹,总共已复制728个
已复制125845_2文件夹,总共已复制729个
已复制125845_5文件夹,总共已复制730个
已复制125845_4文件夹,总共已复制731个
已复制125845_7文件夹,总共已复制732个
已复制125845_6文件夹,总共已复制733个
已复制125845_9文件夹,总共已复制734个
已复制125845_8文件夹,总共已复制735个
已复制125895_1文件夹,总共已复制736个
已复制125895_0文件夹,总共已复制737个
已复制125895_3文件夹,总共已复制738个
已复制125895_2文件夹,总共已复制739个
已复制125895_5文件夹,总共已复制740个
已复制125895_4文件夹,总共已复制741个
已复制125895_7文件夹,总共已

已复制140317_8文件夹,总共已复制1041个
已复制154449_11文件夹,总共已复制1042个
已复制154449_10文件夹,总共已复制1043个
已复制154449_13文件夹,总共已复制1044个
已复制154449_12文件夹,总共已复制1045个
已复制154449_1文件夹,总共已复制1046个
已复制154449_0文件夹,总共已复制1047个
已复制154449_3文件夹,总共已复制1048个
已复制154449_2文件夹,总共已复制1049个
已复制154449_5文件夹,总共已复制1050个
已复制154449_4文件夹,总共已复制1051个
已复制154449_7文件夹,总共已复制1052个
已复制154449_6文件夹,总共已复制1053个
已复制154449_9文件夹,总共已复制1054个
已复制154449_8文件夹,总共已复制1055个
已复制15837_11文件夹,总共已复制1056个
已复制15837_10文件夹,总共已复制1057个
已复制15837_13文件夹,总共已复制1058个
已复制15837_12文件夹,总共已复制1059个
已复制15837_1文件夹,总共已复制1060个
已复制15837_0文件夹,总共已复制1061个
已复制15837_3文件夹,总共已复制1062个
已复制15837_2文件夹,总共已复制1063个
已复制15837_5文件夹,总共已复制1064个
已复制15837_4文件夹,总共已复制1065个
已复制15837_7文件夹,总共已复制1066个
已复制15837_6文件夹,总共已复制1067个
已复制15837_9文件夹,总共已复制1068个
已复制15837_8文件夹,总共已复制1069个
已复制15Wu4U2VPLw_1文件夹,总共已复制1070个
已复制15Wu4U2VPLw_2文件夹,总共已复制1071个
已复制15Wu4U2VPLw_5文件夹,总共已复制1072个
已复制15Wu4U2VPLw_4文件夹,总共已复制1073个
已复制15Wu4U2VPLw_7文件夹,总共已复制1074个
已复制15Wu4U2VPLw_6文件夹,总共已复制1075个
已复制15Wu4U2VPLw_9文件夹,总共已复制1076个
已复制15Wu4U2VPLw_8文件夹,总共已复制1077个
已复制1

已复制1dWsw38VDjY_4文件夹,总共已复制1364个
已复制1dWsw38VDjY_7文件夹,总共已复制1365个
已复制1dWsw38VDjY_6文件夹,总共已复制1366个
已复制1fqNb8cfLFE_0文件夹,总共已复制1367个
已复制1fqNb8cfLFE_2文件夹,总共已复制1368个
已复制1k7br8xXMSU_11文件夹,总共已复制1369个
已复制1k7br8xXMSU_10文件夹,总共已复制1370个
已复制1k7br8xXMSU_13文件夹,总共已复制1371个
已复制1k7br8xXMSU_12文件夹,总共已复制1372个
已复制1k7br8xXMSU_15文件夹,总共已复制1373个
已复制1k7br8xXMSU_1文件夹,总共已复制1374个
已复制1k7br8xXMSU_0文件夹,总共已复制1375个
已复制1k7br8xXMSU_3文件夹,总共已复制1376个
已复制1k7br8xXMSU_5文件夹,总共已复制1377个
已复制1k7br8xXMSU_4文件夹,总共已复制1378个
已复制1k7br8xXMSU_7文件夹,总共已复制1379个
已复制1mHjMNZZvFo_1文件夹,总共已复制1380个
已复制1mHjMNZZvFo_3文件夹,总共已复制1381个
已复制1mHjMNZZvFo_2文件夹,总共已复制1382个
已复制1mHjMNZZvFo_5文件夹,总共已复制1383个
已复制1mHjMNZZvFo_4文件夹,总共已复制1384个
已复制1mHjMNZZvFo_7文件夹,总共已复制1385个
已复制1mHjMNZZvFo_6文件夹,总共已复制1386个
已复制1o8Gy4J6WSs_1文件夹,总共已复制1387个
已复制1o8Gy4J6WSs_3文件夹,总共已复制1388个
已复制1o8Gy4J6WSs_4文件夹,总共已复制1389个
已复制1olbQVO_C-8_1文件夹,总共已复制1390个
已复制1olbQVO_C-8_0文件夹,总共已复制1391个
已复制1olbQVO_C-8_2文件夹,总共已复制1392个
已复制1pvK0_9SQns_10文件夹,总共已复制1393个
已复制1pvK0_9SQns_5文件夹,总共已复制1394个
已复制1pvK0_9SQns_4文件夹,总共已复制1395个
已复

...

已复制d-Uw_uZyUys_0文件夹,总共已复制21553个
已复制d-Uw_uZyUys_2文件夹,总共已复制21554个
已复制d-Uw_uZyUys_7文件夹,总共已复制21555个
已复制d-Uw_uZyUys_6文件夹,总共已复制21556个
已复制d1CDP6sMuLA_11文件夹,总共已复制21557个
已复制d1CDP6sMuLA_10文件夹,总共已复制21558个
已复制d1CDP6sMuLA_12文件夹,总共已复制21559个
已复制d1CDP6sMuLA_15文件夹,总共已复制21560个
已复制d1CDP6sMuLA_14文件夹,总共已复制21561个
已复制d1CDP6sMuLA_19文件夹,总共已复制21562个
已复制d1CDP6sMuLA_18文件夹,总共已复制21563个
已复制d1CDP6sMuLA_1文件夹,总共已复制21564个
已复制d1CDP6sMuLA_3文件夹,总共已复制21565个
已复制d1CDP6sMuLA_4文件夹,总共已复制21566个
已复制d1CDP6sMuLA_6文件夹,总共已复制21567个
已复制d1CDP6sMuLA_9文件夹,总共已复制21568个
已复制d1CDP6sMuLA_8文件夹,总共已复制21569个
已复制dHk--ExZbHs_0文件夹,总共已复制21570个
已复制dHk--ExZbHs_7文件夹,总共已复制21571个
已复制dHk--ExZbHs_6文件夹,总共已复制21572个
已复制dHk--ExZbHs_8文件夹,总共已复制21573个
已复制dR68gbeOWOc_1文件夹,总共已复制21574个
已复制dR68gbeOWOc_3文件夹,总共已复制21575个
已复制dTcz1am1eUw_0文件夹,总共已复制21576个
已复制dTcz1am1eUw_3文件夹,总共已复制21577个
已复制dTcz1am1eUw_2文件夹,总共已复制21578个
已复制ddWHTdJz2O8_1文件夹,总共已复制21579个
已复制ddWHTdJz2O8_0文件夹,总共已复制21580个
已复制ddWHTdJz2O8_2文件夹,总共已复制21581个
已复制dlE05KC95uk_11文件夹,总共已复制21582个
已复制dlE05KC95uk_12文件夹,总共已复制21583个

已复制iPPp6MCythU_1文件夹,总共已复制21811个
已复制iPPp6MCythU_0文件夹,总共已复制21812个
已复制iPPp6MCythU_3文件夹,总共已复制21813个
已复制iPPp6MCythU_7文件夹,总共已复制21814个
已复制iPPp6MCythU_6文件夹,总共已复制21815个
已复制iQDB_OkAQWs_4文件夹,总共已复制21816个
已复制iXiMifHNKPI_10文件夹,总共已复制21817个
已复制iXiMifHNKPI_13文件夹,总共已复制21818个
已复制iXiMifHNKPI_14文件夹,总共已复制21819个
已复制iXiMifHNKPI_0文件夹,总共已复制21820个
已复制iXiMifHNKPI_3文件夹,总共已复制21821个
已复制iXiMifHNKPI_5文件夹,总共已复制21822个
已复制iXiMifHNKPI_7文件夹,总共已复制21823个
已复制iXiMifHNKPI_9文件夹,总共已复制21824个
已复制iXiMifHNKPI_8文件夹,总共已复制21825个
已复制ihPjsliqz5o_9文件夹,总共已复制21826个
已复制ihPjsliqz5o_2文件夹,总共已复制21827个
已复制ihPjsliqz5o_4文件夹,总共已复制21828个
已复制ihPjsliqz5o_7文件夹,总共已复制21829个
已复制j1m6ctAgjsM_42文件夹,总共已复制21830个
已复制j1m6ctAgjsM_22文件夹,总共已复制21831个
已复制j1m6ctAgjsM_43文件夹,总共已复制21832个
已复制j1m6ctAgjsM_23文件夹,总共已复制21833个
已复制j1m6ctAgjsM_24文件夹,总共已复制21834个
已复制j1m6ctAgjsM_25文件夹,总共已复制21835个
已复制j1m6ctAgjsM_26文件夹,总共已复制21836个
已复制j1m6ctAgjsM_27文件夹,总共已复制21837个
已复制j1m6ctAgjsM_20文件夹,总共已复制21838个
已复制j1m6ctAgjsM_21文件夹,总共已复制21839个
已复制j1m6ctAgjsM_48文件夹,总共已复制21840个
已复制j1m6ctAgjsM_49文件夹,总共已复制

已复制mVnqP-vLpuo_9文件夹,总共已复制22091个
已复制mVnqP-vLpuo_8文件夹,总共已复制22092个
已复制mZ_8em_-CGc_4文件夹,总共已复制22093个
已复制mZ_8em_-CGc_7文件夹,总共已复制22094个
已复制mfpR4CN9LZo_24文件夹,总共已复制22095个
已复制mfpR4CN9LZo_20文件夹,总共已复制22096个
已复制mfpR4CN9LZo_23文件夹,总共已复制22097个
已复制mfpR4CN9LZo_16文件夹,总共已复制22098个
已复制mfpR4CN9LZo_0文件夹,总共已复制22099个
已复制mfpR4CN9LZo_4文件夹,总共已复制22100个
已复制mgsvwAVQAQo_1文件夹,总共已复制22101个
已复制mgsvwAVQAQo_0文件夹,总共已复制22102个
已复制mgsvwAVQAQo_3文件夹,总共已复制22103个
已复制mgsvwAVQAQo_5文件夹,总共已复制22104个
已复制mgsvwAVQAQo_4文件夹,总共已复制22105个
已复制mmg_eTDHjkk_20文件夹,总共已复制22106个
已复制mmg_eTDHjkk_21文件夹,总共已复制22107个
已复制mmg_eTDHjkk_22文件夹,总共已复制22108个
已复制mmg_eTDHjkk_1文件夹,总共已复制22109个
已复制mmg_eTDHjkk_0文件夹,总共已复制22110个
已复制mmg_eTDHjkk_3文件夹,总共已复制22111个
已复制mmg_eTDHjkk_2文件夹,总共已复制22112个
已复制mmg_eTDHjkk_5文件夹,总共已复制22113个
已复制mmg_eTDHjkk_4文件夹,总共已复制22114个
已复制mmg_eTDHjkk_7文件夹,总共已复制22115个
已复制mmg_eTDHjkk_6文件夹,总共已复制22116个
已复制mmg_eTDHjkk_9文件夹,总共已复制22117个
已复制mmg_eTDHjkk_8文件夹,总共已复制22118个
已复制mmg_eTDHjkk_11文件夹,总共已复制22119个
已复制mmg_eTDHjkk_10文件夹,总共已复制22120个
已复制mmg_eTDHjkk_13文件夹,总共已复制22121

已复制qEuJj4uW93E_5文件夹,总共已复制22353个
已复制qEuJj4uW93E_4文件夹,总共已复制22354个
已复制qTkazqluJ_I_14文件夹,总共已复制22355个
已复制qTkazqluJ_I_3文件夹,总共已复制22356个
已复制qTkazqluJ_I_2文件夹,总共已复制22357个
已复制qTkazqluJ_I_5文件夹,总共已复制22358个
已复制qTkazqluJ_I_9文件夹,总共已复制22359个
已复制qTkazqluJ_I_8文件夹,总共已复制22360个
已复制qgC8_emxSIU_10文件夹,总共已复制22361个
已复制qgC8_emxSIU_5文件夹,总共已复制22362个
已复制qgC8_emxSIU_6文件夹,总共已复制22363个
已复制qyqVc352g3Q_1文件夹,总共已复制22364个
已复制r46amqjpWgg_24文件夹,总共已复制22365个
已复制r46amqjpWgg_25文件夹,总共已复制22366个
已复制r46amqjpWgg_26文件夹,总共已复制22367个
已复制r46amqjpWgg_27文件夹,总共已复制22368个
已复制r46amqjpWgg_20文件夹,总共已复制22369个
已复制r46amqjpWgg_21文件夹,总共已复制22370个
已复制r46amqjpWgg_22文件夹,总共已复制22371个
已复制r46amqjpWgg_23文件夹,总共已复制22372个
已复制r46amqjpWgg_28文件夹,总共已复制22373个
已复制r46amqjpWgg_29文件夹,总共已复制22374个
已复制r46amqjpWgg_1文件夹,总共已复制22375个
已复制r46amqjpWgg_0文件夹,总共已复制22376个
已复制r46amqjpWgg_3文件夹,总共已复制22377个
已复制r46amqjpWgg_2文件夹,总共已复制22378个
已复制r46amqjpWgg_5文件夹,总共已复制22379个
已复制r46amqjpWgg_4文件夹,总共已复制22380个
已复制r46amqjpWgg_7文件夹,总共已复制22381个
已复制r46amqjpWgg_6文件夹,总共已复制22382个
已复制r46amqjpWgg_9文件夹,总共已复制223

已复制wznRBN1fWj4_6文件夹,总共已复制22630个
已复制wznRBN1fWj4_9文件夹,总共已复制22631个
已复制x0rLwBIocuI_12文件夹,总共已复制22632个
已复制x0rLwBIocuI_4文件夹,总共已复制22633个
已复制x0rLwBIocuI_7文件夹,总共已复制22634个
已复制x266rUJQC_8_10文件夹,总共已复制22635个
已复制x266rUJQC_8_12文件夹,总共已复制22636个
已复制x266rUJQC_8_2文件夹,总共已复制22637个
已复制x266rUJQC_8_5文件夹,总共已复制22638个
已复制x266rUJQC_8_0文件夹,总共已复制22639个
已复制x2n19Cn96aw_9文件夹,总共已复制22640个
已复制x2n19Cn96aw_4文件夹,总共已复制22641个
已复制x8UZQkN52o4_11文件夹,总共已复制22642个
已复制x8UZQkN52o4_10文件夹,总共已复制22643个
已复制x8UZQkN52o4_12文件夹,总共已复制22644个
已复制x8UZQkN52o4_3文件夹,总共已复制22645个
已复制x8UZQkN52o4_2文件夹,总共已复制22646个
已复制x8UZQkN52o4_5文件夹,总共已复制22647个
已复制x8UZQkN52o4_6文件夹,总共已复制22648个
已复制x8UZQkN52o4_9文件夹,总共已复制22649个
已复制xBE9YWYGjtk_1文件夹,总共已复制22650个
已复制xBE9YWYGjtk_0文件夹,总共已复制22651个
已复制xBE9YWYGjtk_3文件夹,总共已复制22652个
已复制xBE9YWYGjtk_2文件夹,总共已复制22653个
已复制xBE9YWYGjtk_5文件夹,总共已复制22654个
已复制xBE9YWYGjtk_4文件夹,总共已复制22655个
已复制xBE9YWYGjtk_7文件夹,总共已复制22656个
已复制xBE9YWYGjtk_6文件夹,总共已复制22657个
已复制xBE9YWYGjtk_9文件夹,总共已复制22658个
已复制xBE9YWYGjtk_8文件夹,总共已复制22659个
已复制xSCvspXYU9k_20文件夹,总共已复制22660个
已

In [86]:
# 处理sims-label
# 打开csv文件
# Load data
dfs = []
for path in glob.glob(sims_label_path):
    dfs.append(pd.read_csv(path))
# Generate table
df1 = pd.concat(dfs)
df1

,video_id,clip_id,text,label,label_T,label_A,label_V,annotation,mode
0,video_0001,1,我不想嫁给李茶,-1.0,-1.0,-1.0,-1.0,Negative,train
1,video_0001,2,你这是嫁入豪门啊！,1.0,1.0,0.8,1.0,Positive,train
2,video_0001,3,我不想嫁入什么豪门，我们不就是豪门吗？,-1.0,-0.8,-1.0,-1.0,Negative,test
3,video_0001,4,有那么明显吗？,-1.0,-0.6,-1.0,-1.0,Negative,train
4,video_0001,5,我在这消费这么多钱，我低血糖吃块糖还不行了,-1.0,-1.0,-1.0,-1.0,Negative,train
...,...,...,...,...,...,...,...,...,...
2276,video_0060,39,看你平常对女生视而不见，没想到不鸣则已一鸣惊人啊,0.2,0.8,0.2,0.4,Positive,valid
2277,video_0060,40,连考了三年都没考上,-0.2,-0.6,-0.2,-0.2,Negative,train
2278,video_0060,41,你怎么来了,0.0,0.0,-0.2,-0.2,Neutral,train
2279,video_0060,42,姐是不是给你丢人了,0.0,-0.8,-0.2,0.6,Neutral,train


In [87]:
# 提取负面文本 POS
# 不含0的负面
# 再取一次不含0的负面
negative_df=df1[df1['annotation']=='Negative']
negative_df=negative_df[['text']]
negative_df

,text
0,我不想嫁给李茶
2,我不想嫁入什么豪门，我们不就是豪门吗？
3,有那么明显吗？
4,我在这消费这么多钱，我低血糖吃块糖还不行了
5,够了，每次都是这一套，我不想再听了。
...,...
2271,我要他先道歉
2272,我告诉你，昨天的事还没完呢
2274,但是别来烦我
2277,连考了三年都没考上


In [89]:
# 保存负面文本
negative_df.to_csv(f'{base_label_output_path}/neg.csv',header=0,index=False)

In [94]:
# 提取正面文本 POS
positive_df=df1[(df1['annotation']=='Positive')]
positive_df=positive_df[['text']]
positive_df

,text
1,你这是嫁入豪门啊！
6,我刚才信号不好，现在可以了，怎么样，妈，公司给我安排的海景别墅楼。
8,说到演讲啊，我不禁想起了我在我的母校。
9,经理经理，别忘了我那五十万和副科长。
10,友德读懂了您对我的暗示。
...,...
2261,他都没正眼看过我
2265,一定要注意身体啊，不要乱吃零食
2267,蜜罐里长大的吧，嘴这么甜
2275,可是我已经投喜欢的人了


In [95]:
# 保存正面文本
positive_df.to_csv(f'{base_label_output_path}/pos.csv',header=0,index=False)